In [ ]:
!python -m pip install --user --upgrade pip

     |████████████████████████████████| 1.5MB 11.5MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 11.8 MB/s 
     |████████████████████████████████| 636 kB 22.3 MB/s 
     |████████████████████████████████| 1.5 MB 33.2 MB/s 
     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 50 kB 3.9 MB/s 
     |████████████████████████████████| 56 kB 2.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 4.1 MB/s 
     |████████████████████████████████| 1.0 MB 53.7 MB/s 
     |████████████████████████████████| 61 kB 5.2 MB/s 
  Created wheel for kfp: filename=kfp-1.4.0-py3-none-any.whl size=221064 sha256=63db60e701d1a3d6a829040430d962705768518127f4db4830717dd85c7efef5
  Stored in directory: /root/.ca

In [ ]:
import kfp
from kfp import dsl
import kfp.components as comp

In [ ]:
def load_data_op():
    return dsl.ContainerOp(
        name = 'Load Data',
        image = 'mavencodevv/load_los:v.0.1',
        arguments = [],
        file_outputs={
            'data': '/load_data/data'
        }      
    )

In [ ]:
def preprocess_op(data):
    return dsl.ContainerOp(
        name = 'Preprocess Data',
        image = 'mavencodevv/preprocess_los:v.0.1',
        arguments = ['--data', data],
        file_outputs={
            'clean_data': '/preprocess/clean_data'   
        }
    )

In [ ]:
def train_op(clean_data):
    return dsl.ContainerOp(
        name = 'Train model',
        image = 'mavencodevv/train_los:v.0.1',
        arguments = ['--clean_data', clean_data
            ],
        file_outputs={
            'fit_model': '/train_model/fit_model' 
        }
    )

In [ ]:
def test_op(clean_data, fit_model):
    return dsl.ContainerOp(
        name = 'Test model',
        image = 'mavencodevv/test_los:v.0.1',
        arguments = ['--clean_data', clean_data,
                    '--fit_model', fit_model
            ],
        file_outputs={
            'metrics': '/test_model/metrics' 
        }
    )

In [ ]:
def results(metrics):
    return dsl.ContainerOp(
        name = 'Results',
        image = 'mavencodevv/results_los:v.0.1',
        arguments = ['--metrics', metrics
            ] 
        
    )

In [ ]:
@dsl.pipeline(
    name='Hospital Length of Stay Pipeline',
   description='An ML reusable pipeline that predicts how long a patient will be admitted in the hospital'
)

# Define parameters to be fed into pipeline
def los_pipeline(): 
    
    _load_data_op = load_data_op()
    
    _preprocess_op = preprocess_op(
        dsl.InputArgumentPath(_load_data_op.outputs['data'])).after(_load_data_op)
    
    _train_op = train_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data'])).after(_preprocess_op)
    
    _test_op = test_op(
        dsl.InputArgumentPath(_preprocess_op.outputs['clean_data']),
        dsl.InputArgumentPath(_train_op.outputs['fit_model'])).after(_train_op)
    _results_op = results(
        dsl.InputArgumentPath(_test_op.outputs['metrics'])).after(_test_op)
    

In [ ]:
# Compile pipeline to generate compressed YAML definition of the pipeline.
experiment_name = 'length_of_stay_pipeline'
#kfp.compiler.Compiler().compile(churn_reuseable_tensorflow_pipeline,  
#  '{}.zip'.format(experiment_name))
kfp.compiler.Compiler().compile(los_pipeline, '{}.yaml'.format(experiment_name))

/root/.local/lib/python3.7/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [ ]:
client = kfp.Client()
client.create_run_from_pipeline_func(ld_pipeline, arguments={})